# 2024-10

In [1]:
import aocd, setup

# Get Puzzle on this Year and Day.
year = 2024
day = 10
session_id = setup.get_sessionid()  # current session id

puzzle = aocd.get_puzzle(session_id, year=year, day=day)            # get puzzle info
data = aocd.get_data(session_id, year=year, day=day).splitlines()   # get input data


In [2]:
# Review the data and examples.
print("Input length:", len(data))
print("Input sample:")

for i in range(7):
    print(data[i])

eg_data = puzzle.examples[0].input_data
print("\nInput example:")
print(eg_data)

eg_answer_a = puzzle.examples[0].answer_a
print("answer a:", eg_answer_a)
eg_answer_b = puzzle.examples[0].answer_b
print("answer b:", eg_answer_b)
eg_extra = puzzle.examples[0].extra
print("extra:", eg_extra)


Input length: 45
Input sample:
012589824304345454345653403898787654310365890
323456712215216783238732312345693201223456781
450144603304306690129641002105892100378901672
343223454456987601234553213456743898465432543
234019765567898501246764789122156784567643456
105678876678983454334895601033021093298932167
987619989569332569543210592344534580112341098

Input example:
89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732
answer a: 36
answer b: None
extra: None


# Part 1

In [27]:
import numpy as np

In [446]:
# Process INPUT data.
input = []
for i in range(len(data)):          # Individualise each digit. 
    input.append(list(data[i]))

map = np.array(input)             # Convert to np.array.
map = map.astype(np.int32)      # Convert to int32.
map.shape

(45, 45)

In [272]:
# Process SAMPLE DATA.
input = eg_data.splitlines()
for i in range(len(input)):
    input[i] = list(input[i])

map = np.array(input)         # Change to np.array.
map = map.astype(np.int32)  # Change type to int32.
map

array([[8, 9, 0, 1, 0, 1, 2, 3],
       [7, 8, 1, 2, 1, 8, 7, 4],
       [8, 7, 4, 3, 0, 9, 6, 5],
       [9, 6, 5, 4, 9, 8, 7, 4],
       [4, 5, 6, 7, 8, 9, 0, 3],
       [3, 2, 0, 1, 9, 0, 1, 2],
       [0, 1, 3, 2, 9, 8, 0, 1],
       [1, 0, 4, 5, 6, 7, 3, 2]], dtype=int32)

In [222]:
# Find starting indices in a map.
def get_starting_indices(map):
    start_idx = np.where(map == 0)                                     # get grid indices for peak = 0.
    start_idx = np.reshape(np.ravel(start_idx, order='F'), (-1,2))     # ravel and reshape the coordinates to get [[x, y]].
    return start_idx


In [259]:
# Climb the map recursively. Return the coordinates of the peak = 9. 
def climb(map, this_row, this_col, peaks):

    # print(map[this_row][this_col], '\t[', this_row, ', ', this_col, ']\t', len(peaks), '\t', peaks)

    if (map[this_row, this_col] == 9):                  # If reached the peak, termininate recursion. 
        return np.array([this_row, this_col])
        
    directions = [[-1, 0], [0, 1], [1, 0], [0, -1]]     # NESW order of direction.

    for r, c in directions:

        if ((this_row + r >= 0) & (this_row + r < len(map))        # Check for map boundaries - row.
            & (this_col + c >= 0) & (this_col + c < len(map[0]))):     # check for map boundaries - col.

            # Condition needed to be separated or it will hit an index out of bounds error.
            if (map[this_row][this_col] +1 == map[this_row + r][this_col + c]):     # Check that height is 1 greater than this location.

                result = climb(map, this_row + r, this_col + c, peaks)      # Get coordinates of '9': recursive results.
                peaks = np.append(peaks, result)                            # Add coordinates to 'peaks' array.
                peaks = np.unique(np.reshape(peaks, (-1, 2)), axis = 0)     # Continually reshape and only keep unique peaks.

    return peaks
            


In [233]:
get_starting_indices(map)

array([[0, 2],
       [0, 4],
       [2, 4],
       [4, 6],
       [5, 2],
       [5, 5],
       [6, 0],
       [6, 6],
       [7, 1]])

In [270]:
# Find the sum of the scores of all trailheads.
start_idxs = get_starting_indices(map)
answer_a = 0

for r, c in start_idxs:
    p = climb(map, r, c, np.array([]))
    answer_a += len(p)

print(answer_a)


538


In [ ]:
# response = aocd.submit(answer_a, part=1, day=day, year=year, session=session_id, reopen=False)

current_day is only available in December (EST)


That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


# Part 2

In [447]:
# Climb the map recursively. Return the coordinates of the peak = 9. 
def get_trails(map, this_row, this_col):

    # print('DIR: [', this_row, ',', this_col, ']\ttop:', map[this_row][this_col])

    if (map[this_row, this_col] == 9):                  # If reached the peak, termininate recursion. 
        return [[this_row, this_col]]
        
    directions = [[-1, 0], [0, 1], [1, 0], [0, -1]]     # NESW order of direction.
    trails = []

    # Travel in all 4 directions.
    for r, c in directions:

        if ((this_row + r >= 0) & (this_row + r < len(map))        # Check for map boundaries - row.
            & (this_col + c >= 0) & (this_col + c < len(map[0]))):     # check for map boundaries - col.

            # Condition needed to be separated or it will hit an index out of bounds error.
            if (map[this_row][this_col] +1 == map[this_row + r][this_col + c]):     # Check that height is 1 greater than this location.

                returned_trails = get_trails(map, this_row + r, this_col + c)       # Get recursive result. 

                if ((returned_trails != []) & (map[this_row][this_col] == 8)):      # If it is level 8, then append to trails array.
                    returned_trails.append([this_row, this_col])
                    trails.append(returned_trails)
                elif ((returned_trails != [])):                                     # If it is level 7 or lower, then append each row to trails array.
                    for i in range(len(returned_trails)):
                        returned_trails[i].append([this_row, this_col])
                        trails.append(returned_trails[i])

                # print('\tBACK: [', this_row, ',', this_col ,']\tlen:', len(returned_trails),'\ttop:', map[this_row][this_col],'\t', returned_trails)
                    
    # print('\t\tBACK2: [', this_row, ',', this_col ,']\tlen:', len(trails),'\ttop:', map[this_row][this_col],'\t', trails, '\n')
    return trails
            


In [448]:
# Calculate the total sum rating. 
start_idx = get_starting_indices(map)
start_idx

sum_rating = 0

for r,c in start_idx:
    trails = get_trails(map, r, c)
    sum_rating += len(trails)

    print(r,c,len(trails), sum_rating)


0 0 2 2
0 10 21 23
0 25 2 25
0 38 2 27
0 44 5 32
1 34 3 35
2 2 2 37
2 7 11 48
2 10 21 69
2 13 1 70
2 17 6 76
2 24 1 77
2 25 3 80
2 28 1 81
2 34 3 84
2 35 3 87
2 40 6 93
3 16 1 94
4 3 4 98
4 16 1 99
5 1 2 101
5 25 4 105
5 27 8 113
5 30 2 115
5 33 3 118
6 23 8 126
6 35 2 128
6 42 12 140
7 4 11 151
7 11 4 155
7 15 5 160
7 37 3 163
8 1 6 169
8 5 18 187
8 13 2 189
8 15 10 199
8 17 5 204
8 19 10 214
8 33 3 217
9 14 5 222
9 20 10 232
9 39 2 234
10 1 22 256
10 19 3 259
10 27 4 263
10 33 3 266
10 41 7 273
10 42 1 274
11 10 4 278
11 11 2 280
11 36 14 294
12 2 1 295
12 7 4 299
12 12 4 303
12 22 3 306
12 26 4 310
12 30 2 312
13 4 5 317
13 8 4 321
13 13 2 323
13 18 3 326
13 28 1 327
13 44 4 331
14 16 2 333
14 22 5 338
14 33 6 344
14 41 4 348
15 3 1 349
15 28 4 353
15 39 2 355
15 44 4 359
16 5 7 366
16 7 12 378
16 19 2 380
16 21 3 383
16 22 4 387
17 10 5 392
17 11 1 393
17 27 3 396
17 32 8 404
17 34 8 412
17 36 6 418
18 1 19 437
18 15 8 445
18 27 1 446
18 38 1 447
18 39 1 448
18 42 4 452
19 2 2 454


In [449]:
sum_rating

1110

In [450]:
response = aocd.submit(sum_rating, part=2, day=day, year=year, session=session_id, reopen=False)

current_day is only available in December (EST)


That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 10! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
